Instalando Pacotes

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas

Função + extraindo cidades

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

url = "https://listacep.com/ac/acrelandia/centro/avenida-parana-296/69945970"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# Função para extração de links
def extract_links(url):
    # fazendo o GET
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    empty_div = soup.find('div', class_='list__empty')
    # Se encontrar a classe list__empty é pra ele retornar None
    if empty_div:
        print(f"Não há informações de links para {url}")
        return None
    else:
        # se não é pra ele puxar as informações da lista
        primeira_ul = soup.find('ul', class_='list__list')

        if primeira_ul:
            links = [a['href'] for a in primeira_ul.find_all('a')]
            return links
        else:
            return []

# link do site
url = "https://listacep.com/"

# extração dos estados
estados_links = extract_links(url)

# extração das cidades
cidades_links = []
for estado_link in estados_links:
    cidades_links.extend(extract_links(estado_link))

# caso ele encontre 2x a mesma informação ou repita algo que está na lista de estados por algum erro
cidades_links = list(set(cidades_links)) 
cidades_links = [cidade for cidade in cidades_links if cidade not in estados_links]
cidades_links

Extraindo bairros

In [ ]:
bairros_links = []
max_attempts = 5

cidades_sem_bairros = set()

for cidade_link in cidades_links:

    if cidade_link in cidades_sem_bairros:
        continue

    attempts = 0
    while attempts < max_attempts:
        try:
            bairros = extract_links(cidade_link)
            
            if bairros is None:
                cidades_sem_bairros.add(cidade_link)  
                break  

            bairros_links.extend(bairros)
            break
        except Exception as e:
            print(f"Erro ao processar {cidade_link}. Tentando novamente... ({attempts+1}/{max_attempts})")
            attempts += 1

bairros_links = list(set(bairros_links))
bairros_links = [bairro for bairro in bairros_links if bairro not in cidades_links]
bairros_links

Extraindo Ruas

In [ ]:
ruas_links = []
max_attempts = 5

bairros_sem_ruas = set()

for bairro_link in bairros_links:

    if bairro_link in bairros_sem_ruas:
        continue

    attempts = 0
    while attempts < max_attempts:
        try:
            ruas = extract_links(bairro_link)
            
            if ruas is None:
                bairros_sem_ruas.add(bairro_link)  
                break  

            ruas_links.extend(ruas)
            break
        except Exception as e:
            print(f"Erro ao processar {ruas_links}. Tentando novamente... ({attempts+1}/{max_attempts})")
            attempts += 1

ruas_links = list(set(ruas_links))
ruas_links = [rua for rua in ruas_links if rua not in bairros_links]
ruas_links